<a href="https://colab.research.google.com/github/kumaradityaka/Recommendation_Bot/blob/main/Recommendation_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import numpy as np
import json
import re
import tensorflow as tf
import random
import spacy
nlp = spacy.load('en_core_web_sm')

In [25]:
with open('/content/drive/MyDrive/Data/Intent.json') as f:
    intents = json.load(f)

In [26]:
def preprocessing(line):
    line = re.sub(r'[^a-zA-z.?!\']', ' ', line)
    line = re.sub(r'[ ]+', ' ', line)
    return line
  

In [27]:

inputs, targets = [], []
classes = []
intent_doc = {}

for intent in intents['intents']:
    if intent['intent'] not in classes:
        classes.append(intent['intent'])
    if intent['intent'] not in intent_doc:
        intent_doc[intent['intent']] = []
        
    for text in intent['text']:
        inputs.append(preprocessing(text))
        targets.append(intent['intent'])
        
    for response in intent['responses']:
        intent_doc[intent['intent']].append(response)

In [28]:
def tokenize_data(input_list):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<unk>')
    
    tokenizer.fit_on_texts(input_list)
    
    input_seq = tokenizer.texts_to_sequences(input_list)

    input_seq = tf.keras.preprocessing.sequence.pad_sequences(input_seq, padding='pre')
    
    return tokenizer, input_seq

# preprocess input data
tokenizer, input_tensor = tokenize_data(inputs)


In [29]:
def create_categorical_target(targets):
    word={}
    categorical_target=[]
    counter=0
    for trg in targets:
        if trg not in word:
            word[trg]=counter
            counter+=1
        categorical_target.append(word[trg])
    
    categorical_tensor = tf.keras.utils.to_categorical(categorical_target, num_classes=len(word), dtype='int32')
    return categorical_tensor, dict((v,k) for k, v in word.items())

# preprocess output data
target_tensor, trg_index_word = create_categorical_target(targets)

In [30]:
print('input shape: {} and output shape: {}'.format(input_tensor.shape, target_tensor.shape))

input shape: (128, 7) and output shape: (128, 20)


In [31]:
# hyperparameters
epochs=50
vocab_size=len(tokenizer.word_index) + 1
embed_dim=512
units=128
target_length=target_tensor.shape[1]

In [32]:
# build RNN Model with tensorflow
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(vocab_size, embed_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units, dropout=0.2)),
    tf.keras.layers.Dense(units, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(target_length, activation='softmax')
])

optimizer = tf.keras.optimizers.Adam(lr=1e-2)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 512)         59392     
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               656384    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 20)                2580      
Total params: 751,252
Trainable params: 751,252
Non-trainable params: 0
_________________________________________________________________


In [33]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4)

# train the model
model.fit(input_tensor, target_tensor, epochs=55)

Epoch 1/55
4/4 [==============================] - 3s 56ms/step - loss: 3.0030 - accuracy: 0.0542
Epoch 2/55
4/4 [==============================] - 0s 59ms/step - loss: 2.6809 - accuracy: 0.2490
Epoch 3/55
4/4 [==============================] - 0s 61ms/step - loss: 1.8949 - accuracy: 0.5396
Epoch 4/55
4/4 [==============================] - 0s 59ms/step - loss: 1.1110 - accuracy: 0.6844
Epoch 5/55
4/4 [==============================] - 0s 61ms/step - loss: 0.7236 - accuracy: 0.7760
Epoch 6/55
4/4 [==============================] - 0s 56ms/step - loss: 0.3913 - accuracy: 0.8771
Epoch 7/55
4/4 [==============================] - 0s 64ms/step - loss: 0.2661 - accuracy: 0.9146
Epoch 8/55
4/4 [==============================] - 0s 56ms/step - loss: 0.2220 - accuracy: 0.9292
Epoch 9/55
4/4 [==============================] - 0s 58ms/step - loss: 0.0874 - accuracy: 0.9823
Epoch 10/55
4/4 [==============================] - 0s 58ms/step - loss: 0.0732 - accuracy: 0.9917
Epoch 11/55
4/4 [============

In [34]:
def response(sentence):
    sent_seq = []
    doc = nlp(repr(sentence))
    
    # split the input sentences into words
    for token in doc:
        if token.text in tokenizer.word_index:
            sent_seq.append(tokenizer.word_index[token.text])

        # handle the unknown words error
        else:
            sent_seq.append(tokenizer.word_index['<unk>'])

    sent_seq = tf.expand_dims(sent_seq, 0)
    # predict the category of input sentences
    pred = model(sent_seq)

    pred_class = np.argmax(pred.numpy(), axis=1)
     # choice a random response for predicted sentence
    return random.choice(intent_doc[trg_index_word[pred_class[0]]]), trg_index_word[pred_class[0]]

# chat with bot
print("Note: Genres the bot can recommend about are : Music, Games, Comedy")
print("Note: Enter 'Recommed me (genres) videos or (genres) videos or (genres)' to get recommendation based on genres.")
print("For example: 'Recommend me music videos' ")
print("Note: Enter 'quit' to break the loop.")
while True:
    input_ = input('You: ')
    if input_.lower() == 'quit':
        break
    res, typ = response(input_.lower())
    print('Bot: {} -- TYPE: {}'.format(res, typ))
    print()

Note: Genres the bot can recommend about are : Music, Games, Comedy
Note: Enter 'Recommed me (genres) videos or (genres) videos or (genres)' to get recommendation based on genres.
For example: 'Recommend me music videos' 
Note: Enter 'quit' to break the loop.
You: Hello
Bot: Hi, I am great, how are you?  -- TYPE: CourtesyGreeting

You: Great
Bot: Great! Hi <HUMAN>! How can I help? -- TYPE: CourtesyGreetingResponse

You: Recommend me games
Bot: https://www.youtube.com/watch?v=stFgmKAU2B4 -- TYPE: RecommendGames

You: quit
